# Machine Learning in spectroscopy

This notebook presents the data that you will be using for the different projects. In this laboratory we are trying to predict different aerosols' concentration from absorption spectra. These spectra represent the infrared light absorption between $4000 [cm^{-1}]$ and $400 [cm^{-1}]$.

The aerosolos are collected on polytetrafluoroethylene (PTFE) filters (i.e. commonly known as Teflon) and the spectra are obtained through the Fourier Transorm InfraRed (FTIR) spectroscopy technique.

During this project, you will be using the IMPROVE 2015 dataset. IMPROVE is a network of monitoring stations collecting information about atmospheric aerosols all over the United-States but mostly in rural areas. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

## Data

There are three files containing different information. The first, `IMPROVE-2015_spectra.csv`, contains FTIR absorption spectra of Teflon filters taken at different locations. The second, `IMPROVE-2015_measures.csv`, contains the measures of different species.
The last one contains the usage of a site, if it has been used for training or testing purposes.

In [ ]:
filename_measures = 'data/IMPROVE_2015_measures_cs433.csv'
filename_spectra = 'data/IMPROVE_2015_raw_spectra_cs433.csv'
filename_tts = 'data/IMPROVE_2015_train_test_split_cs433.csv'
filename_sec_deriv = 'data/IMPROVE_2015_2nd-derivative_spectra_cs433.csv'

df_meas = pd.read_csv(filename_measures)
df_spec = pd.read_csv(filename_spectra)
df_tts = pd.read_csv(filename_tts)
df_sec_deriv = pd.read_csv(filename_sec_deriv, index_col=0)

In [ ]:
df_spec.head()

Each sample is represented by 2784 wavenumber. The sites have a flag at the end of their name, here are their description:

- NM: Normal sample – this corresponds to ambient samples whose data (flow, mass, etc… ) have been validated and no abnormalities have been noticed (bulk of the data)
- FB: Field blanks samples
- QD: Questionable Data – those are samples which were flagged during the validation stage as potential bad samples. If QD samples made it to the FTIR lab, however, it usually means that the validation group changed the flag status from QD to NM after careful consideration. In summary, you can use those samples in your model (no issues observed so far).
- QC: Quality Control – Unlike previous flags, this one is specific to our group (FT-IR). QC samples are a group of 10 randomly picked ambient samples measured on a single instrument that were re-run on another spectrometer to provide inter-instrument comparison as part of our weekly quality control procedure. Considering that our instruments show little spectral dissimilarities between them, QC samples can almost be seen as duplicates.

The samples present in the dataset correspond to a measures every 3 days in a 24-hour period. If there are some samples missing (i.e. a date when there should be a sample), it means that the measure was not validated.

In [ ]:
df_meas.head()

The tasks are to predict different species. The relevant columns for your work are the following:

- SiteCode: site ID
- Date: date of the measure
- TOR OC: `OCf:Value`
- TOR EC: `ECf:Value`
- Dust: `DUSTf:Value`
- Sulfate: `SO4f:Value`
- Nitrate: `NO3f:Value`
- PM2.5: `MF:Value`

They all have an uncertainty value with the suffix `:Unc`.

In [ ]:
df_tts.head()

In [ ]:
x = df_spec['wavenumber']
y = df_spec.loc[:, df_spec.columns != 'wavenumber']

plt.figure(figsize=(10,6))
plt.plot(x, y)
plt.xlabel('Wavenumber')
plt.ylabel('Absorption')
ax = plt.gca()
ax.invert_xaxis()
plt.show()

## Feature processing

### Baseline correction
The main feature processing is baseline correction, in other words, removing the background of the signal. There are two different processes that you could use to remove the baseline: smoothing spline and second derivatives. If you are interested in the techniques themselves, you can read about how we applied smoothing spline in our baseline correction process in the paper "An automated baseline correction protocol for infrared spectra of atmospheric aerosols collected on polytetrafluoroethylene (Teflon) filters" (Kuzmiakova, Dillner, and Takahama, Atmos. Meas. Tech., 2016).

We will provide you the datasets prepared in the case you want to use them in your project.

Here are the options used to produce the datasets:

- **Second derivative**: second-derivative spectra generated with a symmetric, second-order, 21-point Savitsky-Golay filter ("Ambient aerosol composition by infrared spectroscopy and partial least-squares in the chemical speciation network: Organic carbon with functional group identification". Weakly, Takahama, and Dillner, Aeros. Science and Technology. 2016)


In [ ]:
# 2nd derivative data plot
x = df_sec_deriv['wavenumber']
y = df_sec_deriv.iloc[:, df_sec_deriv.columns != 'wavenumber']

plt.figure(figsize=(10,6))
plt.plot(x, y)
plt.xlabel('Wavenumber')
plt.ylabel('Absorption')
ax = plt.gca()
ax.invert_xaxis()
plt.show()

### Segmentation
Another feature processing we found useful particulary during baseline correction is segmentation. We split the spectra into three bins: 4000-2000 $[cm^{-1}]$, 2000-1495 $[cm^{-1}]$, and 2000-680 $[cm^{-1}]$. You may also find the segmentation useful.

## Current results

One of our model is Convolutional Neural Network that uses raw spectra as features and predicts different concentrations, here Organic Carbon.

![cnn_res](figures/IMPROVE-2015_CNN_Predictions.png)

## Evaluation

The evaluation will be based on mean error, mean bias, coefficient of determination $r^2$ (i.e. r-squared) and compare against TOR precision. You will have to provide a "cross plot" (prediction vs. reference) and also the normalized residual vs. S/N ratio plot to show there isn't a strong or systematic bias at low concentrations.